In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch.utils.data import DataLoader
from dataset import dataset_hparams, MaestroDataset
from model import model_hparams, Model

In [2]:
maestro_dataset = MaestroDataset(dataset_hparams)
print(maestro_dataset)

train_loader = DataLoader(maestro_dataset, batch_size=16)
print(train_loader)

In [3]:
def tokens_to_event_list(tokens, hp):
    current_time = 0
    current_velocity = 0
    event_list = []
    for token in tokens:
        # interval
        if token < hp.offsets.velocity:
            current_time += token / hp.dims.interval * hp.max_note_duration
            
        # velocity
        elif token < hp.offsets.note_on:
            current_velocity = (token - hp.offsets.velocity) / hp.dims.velocity * 128
            
        # note_on
        elif token < hp.offsets.note_off:
            event = {'time': current_time,
                     'type': 'note_on',
                     'note': token - hp.offsets.note_on,
                     'velocity': int(current_velocity)}
            event_list.append(event)
                     
        # note_off
        elif token < hp.offsets.pedal_on:
            event = {'time': current_time,
                     'type': 'note_off',
                     'note': token - hp.offsets.note_off}
            event_list.append(event)
            
        # pedal_on
        elif token < hp.offsets.pedal_off:
            event = {'time': current_time,
                     'type': 'pedal_on'}
            event_list.append(event)
            
        # pedal_off
        else:
            event = {'time': current_time,
                     'type': 'pedal_off'}
            event_list.append(event)
    
    return event_list
    

In [4]:
def save_event_list_to_midi_file(event_list, midi_file):
    
    new_event_list = []
    notes = [None for _ in range(128)]
    for event in event_list:
        if event['type'] == 'note_on':
            notes[event['note']] = event
        elif event['type'] == 'note_off' and notes[event['note']] is not None:
            notes[event['note']]['duration'] = max(event['time'] - notes[event['note']]['time'], 0)
            new_event_list.append(notes[event['note']])
            notes[event['note']] = None
        elif event['type'] == 'pedal_on' or event['type'] == 'pedal_off': 
            new_event_list.append(event)
    event_list = new_event_list
    event_list.sort(key=lambda x:x['time'])
#     for event in event_list:
#         print(event)
            
    from midiutil import MIDIFile
    '''
    Reference : https://midiutil.readthedocs.io/en/1.2.1/index.html
    '''
    track    = 0
    channel  = 0
    tempo    = 60  # In BPM
    MyMIDI = MIDIFile(1) # One track, defaults to format 1 (tempo track
                         # automatically created)
    MyMIDI.addTempo(track, 0, tempo)
    
    for event in event_list:
        if event['type'] == 'note_on':
            MyMIDI.addNote(track, channel, event['note'], event['time'], event['duration'], event['velocity'])
            
        elif event['type'] == 'pedal_on':
            MyMIDI.addControllerEvent(track, channel, event['time'], 64, 1)
            
        elif event['type'] == 'pedal_off':
            MyMIDI.addControllerEvent(track, channel, event['time'], 64, 0)
            
    with open(midi_file, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    

In [50]:
event_list = tokens_to_event_list(maestro_dataset[1][:40], dataset_hparams)
for event in event_list:
    print(event)

{'time': 0.0, 'type': 'note_on', 'note': 50, 'velocity': 76}
{'time': 0.0, 'type': 'note_off', 'note': 62}
{'time': 0.02, 'type': 'note_off', 'note': 50}
{'time': 0.04, 'type': 'note_on', 'note': 78, 'velocity': 80}
{'time': 0.04, 'type': 'note_off', 'note': 81}
{'time': 0.04, 'type': 'note_off', 'note': 78}
{'time': 0.08, 'type': 'note_on', 'note': 61, 'velocity': 92}
{'time': 0.08, 'type': 'note_on', 'note': 81, 'velocity': 84}
{'time': 0.08, 'type': 'note_on', 'note': 51, 'velocity': 72}
{'time': 0.08, 'type': 'note_off', 'note': 61}
{'time': 0.08, 'type': 'note_off', 'note': 51}
{'time': 0.12, 'type': 'note_on', 'note': 85, 'velocity': 80}
{'time': 0.12, 'type': 'note_off', 'note': 81}
{'time': 0.18, 'type': 'note_on', 'note': 93, 'velocity': 92}
{'time': 0.18, 'type': 'note_on', 'note': 57, 'velocity': 92}


In [51]:
save_event_list_to_midi_file(event_list, 'sample.mid')

KeyError: '780'

In [52]:
!git clone https://github.com/louisabraham/python3-midi.git

Cloning into 'python3-midi'...
remote: Enumerating objects: 553, done.
remote: Total 553 (delta 0), reused 0 (delta 0), pack-reused 553
Receiving objects: 100% (553/553), 207.89 KiB | 4.83 MiB/s, done.
Resolving deltas: 100% (315/315), done.


In [55]:
!cd python3-midi && python setup.py install

/home/gaudio/anaconda3/envs/ste/lib/python3.8/site-packages/setuptools/dist.py:484: UserWarning: Normalizing 'v0.2.3' to '0.2.3'
  warnings.warn(tmpl.format(**locals()))
running install
running bdist_egg
running egg_info
creating midi.egg-info
writing midi.egg-info/PKG-INFO
writing dependency_links to midi.egg-info/dependency_links.txt
writing top-level names to midi.egg-info/top_level.txt
writing manifest file 'midi.egg-info/SOURCES.txt'
reading manifest file 'midi.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'midi.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/midi
copying src/__init__.py -> build/lib/midi
copying src/containers.py -> build/lib/midi
copying src/events.py -> build/lib/midi
copying src/util.py -> build/lib/midi
copying src/fileio.py -> build/lib/midi
copying src/constants.py -> build/lib/midi
creating build/bdist.linux-x8

In [1]:
import midi
# Instantiate a MIDI Pattern (contains a list of tracks)
pattern = midi.Pattern()
# Instantiate a MIDI Track (contains a list of MIDI events)
track = midi.Track()
# Append the track to the pattern
pattern.append(track)
# Instantiate a MIDI note on event, append it to the track
on = midi.NoteOnEvent(tick=0, velocity=20, pitch=midi.G_3)
track.append(on)
# Instantiate a MIDI note off event, append it to the track
off = midi.NoteOffEvent(tick=100, pitch=midi.G_3)
track.append(off)
# Add the end of track event, append it to the track
eot = midi.EndOfTrackEvent(tick=1)
track.append(eot)
# Print out the pattern
print(pattern)
# Save the pattern to disk
midi.write_midifile("example.mid", pattern)

midi.Pattern(format=1, resolution=220, tracks=\
[midi.Track(\
  [midi.NoteOnEvent(tick=0, channel=0, data=[43, 20]),
   midi.NoteOffEvent(tick=100, channel=0, data=[43, 0]),
   midi.EndOfTrackEvent(tick=1, data=[])])])
